In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

/home/ahmad/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Supaya hasil acaknya sama terus
np.random.seed(42)

# Bikin 1000 Data Pelanggan
n_samples = 1000

data = {
    'Tenure': np.random.randint(1, 72, n_samples), # 1 s/d 72 bulan
    'MonthlyCharges': np.random.randint(20, 100, n_samples), # 20 s/d 100 dollar
    'PaperlessBilling': np.random.randint(0, 2, n_samples),
    'SeniorCitizen': np.random.randint(0, 2, n_samples),
    'Partner': np.random.randint(0, 2, n_samples),
    'Dependents': np.random.randint(0, 2, n_samples),
    'PhoneService': np.random.randint(0, 2, n_samples),
    'LuckyNumber': np.random.randint(1, 1000, n_samples) # Fitur Sampah (Random)
}

df = pd.DataFrame(data)

# Bikin kolom TotalCharges (Tenure * Monthly) + sedikit acak
df['TotalCharges'] = df['Tenure'] * df['MonthlyCharges']

# Bikin Target CHURN (Logika: Kalau Tenure kecil & Tagihan Mahal -> Cenderung Churn)
# Ini rumus rahasia biar ada polanya
score = (100 - df['Tenure']) + (df['MonthlyCharges'] * 0.5) + np.random.normal(0, 20, n_samples)
df['Churn'] = (score > 80).astype(int) # 1 = Churn, 0 = Setia
df

,Tenure,MonthlyCharges,PaperlessBilling,SeniorCitizen,Partner,Dependents,PhoneService,LuckyNumber,TotalCharges,Churn
0,52,65,0,1,0,1,1,963,3380,0
1,15,35,0,0,0,0,0,193,525,1
2,61,87,0,0,0,0,0,895,5307,0
3,21,56,1,1,1,0,1,58,1176,1
4,24,73,0,1,1,0,1,437,1752,1
...,...,...,...,...,...,...,...,...,...,...
995,51,99,1,1,0,0,1,352,5049,1
996,8,26,0,0,0,1,1,701,208,1
997,34,71,1,1,1,1,0,928,2414,1
998,35,33,0,0,1,0,1,256,1155,1


In [10]:
x = df.drop(columns='Churn')
y = df['Churn']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
selector_filter = SelectKBest(score_func=chi2, k=3)

#fit dan transform 
x_train_filter = selector_filter.fit_transform(x_train, y_train)
x_test_filter = selector_filter.transform(x_test)

cols_filter = selector_filter.get_support(indices=True)
x.columns[cols_filter]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [50]:
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model_lr = LogisticRegression(solver='liblinear')

#siapkan rfe
selector_rfe = RFE(estimator=model_lr, n_features_to_select=3)

#fit dan transform
x_train_rfe = selector_rfe.fit_transform(x_train_scaled, y_train)
x_test_rfe = selector_rfe.transform(x_test_scaled)

cols_rfe = selector_rfe.get_support(indices=True)
x.columns[cols_rfe]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [43]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(x_train, y_train)

#ambil skor kepentingan
importance = model_rf.feature_importances_
indices_rf = np.argsort(importance)[::-1][:3]
indices_rf

array([0, 8, 1])

In [48]:
x_train_emb = x_train.iloc[:, indices_rf]
x_test_emb = x_test.iloc[:, indices_rf]
x_train_emb

,Tenure,TotalCharges,MonthlyCharges
29,64,3584,56
535,15,465,31
695,7,518,74
557,17,918,54
836,6,270,45
...,...,...,...
106,15,885,59
270,46,1288,28
860,47,3102,66
435,37,2664,72


# SKENARIO 2

In [52]:
import pandas as pd
import numpy as np

np.random.seed(42)
n = 1000

# Membuat Data Campuran
data = {
    'Satisfaction': np.random.rand(n), # Angka 0 sampai 1
    'LastEval': np.random.rand(n),     # Angka 0 sampai 1
    'Projects': np.random.randint(2, 8, n), # 2 sampai 7 proyek
    'Salary': np.random.choice(['Low', 'Medium', 'High'], n), # TEKS
    'Dept': np.random.choice(['Sales', 'IT', 'HR'], n)        # TEKS
}

df_hr = pd.DataFrame(data)

# Logika Target (Resign):
# Orang Resign biasanya: Gaji Rendah (Low) DAN Kerja Banyak (Projects > 5) DAN Kepuasan Rendah
# Kita buat logika matematikanya:
conditions = (
    (df_hr['Salary'] == 'Low') & 
    (df_hr['Projects'] > 5) & 
    (df_hr['Satisfaction'] < 0.5)
)
# Jika kondisi terpenuhi, peluang resign tinggi (80%), jika tidak kecil (10%)
df_hr['Left'] = np.where(conditions, 
                      np.random.choice([1, 0], n, p=[0.9, 0.1]), 
                      np.random.choice([1, 0], n, p=[0.1, 0.9]))

print("Data Mentah (Masih ada Teks):")
display(df_hr.head())

Data Mentah (Masih ada Teks):


,Satisfaction,LastEval,Projects,Salary,Dept,Left
0,0.374540,0.185133,5,Medium,Sales,0
1,0.950714,0.541901,4,Low,HR,0
2,0.731994,0.872946,5,Low,Sales,0
3,0.598658,0.732225,7,High,Sales,0
4,0.156019,0.806561,3,Low,IT,0


In [53]:
df_hr

,Satisfaction,LastEval,Projects,Salary,Dept,Left
0,0.374540,0.185133,5,Medium,Sales,0
1,0.950714,0.541901,4,Low,HR,0
2,0.731994,0.872946,5,Low,Sales,0
3,0.598658,0.732225,7,High,Sales,0
4,0.156019,0.806561,3,Low,IT,0
...,...,...,...,...,...,...
995,0.091582,0.656955,2,Medium,HR,0
996,0.917314,0.956615,2,Low,HR,0
997,0.136819,0.068958,7,Low,HR,1
998,0.950237,0.057055,4,High,IT,0


In [87]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
le = LabelEncoder()

df_encoded['Salary'] = le.fit_transform(df['Salary'])
df_encoded['Dept'] = le.fit_transform(df['Dept'])

x = df_encoded.drop(columns='Left')
y = df_encoded['Left']

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


selector = SelectKBest(score_func=chi2, k=3)

x_new = selector.fit_transform(x, y)
selected_cols = selector.get_support(indices=True)

x.columns[selected_cols]

Index(['Satisfaction', 'Projects', 'Salary'], dtype='object')

In [105]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

model_lr = LogisticRegression(solver='liblinear')
rfe = RFE(estimator=model_lr, n_features_to_select=3)

rfe.fit(x_scaled, y)
rfe_cols = rfe.get_support(indices=True)
x.columns[rfe_cols]

Index(['Satisfaction', 'LastEval', 'Projects'], dtype='object')

In [113]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x, y)


importances = rf.feature_importances_
top3_indices = np.argsort(importances[::-1][:3])
x.columns[top3_indices]

Index(['Satisfaction', 'Projects', 'LastEval'], dtype='object')

# Transaksi Bodong atau Asli

In [179]:
import pandas as pd
import numpy as np

# Agar hasil random kita sama
np.random.seed(123)
n_samples = 1000

# 1. GENERATE DATA
data = {
    'TransactionAmt': np.random.randint(10, 5000, n_samples), # Jumlah Transaksi ($10 - $5000)
    'ProductCD': np.random.choice(['W', 'C', 'R', 'H', 'S'], n_samples), # Kode Produk (Kategori)
    'Card_Type': np.random.choice(['debit', 'credit'], n_samples), # Jenis Kartu
    'P_emaildomain': np.random.choice(['gmail.com', 'yahoo.com', 'hotmail.com', 'anon.com'], n_samples), # Email
    'Device_Info': np.random.choice(['Samsung', 'iOS', 'Windows', 'Mac'], n_samples), # Info HP/Laptop
    'Distance_From_Home': np.random.randint(1, 500, n_samples), # Jarak transaksi dari rumah (KM)
    'Lucky_User_ID': np.random.randint(1000, 9999, n_samples) # ID User (Jebakan/Noise)
}

df_fraud = pd.DataFrame(data)

# 2. MEMBUAT LOGIKA TARGET (FRAUD = 1)
# Logika: Fraud biasanya terjadi jika Transaksi Besar ATAU Jarak Jauh ATAU pakai email anonim
conditions = (
    (df_fraud['TransactionAmt'] > 3000) | 
    (df_fraud['Distance_From_Home'] > 300) | 
    (df_fraud['P_emaildomain'] == 'anon.com')
)
# Beri label 1 (Fraud) jika kondisi terpenuhi (peluang 70%), sisanya 0
df_fraud['is_Fraud'] = np.where(conditions, 
                                np.random.choice([1, 0], n_samples, p=[0.7, 0.3]), 
                                np.random.choice([1, 0], n_samples, p=[0.1, 0.9]))

# 3. MENGOTORI DATA (Membuat Missing Values / NaN)
# Kita hapus sebagian data di kolom 'Device_Info' dan 'P_emaildomain'
idx_nan_device = np.random.choice(n_samples, 200, replace=False) # 200 data hilang
idx_nan_email = np.random.choice(n_samples, 50, replace=False)   # 50 data hilang

df_fraud.loc[idx_nan_device, 'Device_Info'] = np.nan
df_fraud.loc[idx_nan_email, 'P_emaildomain'] = np.nan

print("Data Transaksi Siap (Banyak PR-nya!):")
display(df_fraud.head(10))
print("\nInfo Data Kosong:")
print(df_fraud.isnull().sum())

Data Transaksi Siap (Banyak PR-nya!):


,TransactionAmt,ProductCD,Card_Type,P_emaildomain,Device_Info,Distance_From_Home,Lucky_User_ID,is_Fraud
0,3592,H,credit,gmail.com,iOS,15,1189,0
1,3464,H,credit,anon.com,Windows,402,6339,1
2,1356,C,debit,hotmail.com,Mac,52,4895,0
3,4070,W,credit,yahoo.com,Windows,111,8526,1
4,1603,W,debit,hotmail.com,Windows,481,4017,1
5,106,S,credit,hotmail.com,Samsung,195,6962,0
6,4153,C,debit,gmail.com,Mac,463,4741,0
7,4179,H,credit,anon.com,NaN,81,3577,1
8,952,S,debit,anon.com,Samsung,380,1866,1
9,121,R,credit,anon.com,iOS,485,4830,1



Info Data Kosong:
TransactionAmt          0
ProductCD               0
Card_Type               0
P_emaildomain          50
Device_Info           200
Distance_From_Home      0
Lucky_User_ID           0
is_Fraud                0
dtype: int64


In [180]:
df_fraud

,TransactionAmt,ProductCD,Card_Type,P_emaildomain,Device_Info,Distance_From_Home,Lucky_User_ID,is_Fraud
0,3592,H,credit,gmail.com,iOS,15,1189,0
1,3464,H,credit,anon.com,Windows,402,6339,1
2,1356,C,debit,hotmail.com,Mac,52,4895,0
3,4070,W,credit,yahoo.com,Windows,111,8526,1
4,1603,W,debit,hotmail.com,Windows,481,4017,1
...,...,...,...,...,...,...,...,...
995,2661,W,credit,anon.com,Windows,411,8520,1
996,1719,H,credit,hotmail.com,iOS,29,6412,0
997,4063,R,debit,hotmail.com,iOS,441,8797,1
998,3468,W,debit,hotmail.com,Mac,218,5308,1


In [181]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TransactionAmt      1000 non-null   int64 
 1   ProductCD           1000 non-null   object
 2   Card_Type           1000 non-null   object
 3   P_emaildomain       950 non-null    object
 4   Device_Info         800 non-null    object
 5   Distance_From_Home  1000 non-null   int64 
 6   Lucky_User_ID       1000 non-null   int64 
 7   is_Fraud            1000 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 62.6+ KB


In [183]:
df_fraud[df_fraud.isnull().any(axis=1)]

,TransactionAmt,ProductCD,Card_Type,P_emaildomain,Device_Info,Distance_From_Home,Lucky_User_ID,is_Fraud
7,4179,H,credit,anon.com,NaN,81,3577,1
10,3491,H,credit,NaN,iOS,15,5501,1
14,1102,C,debit,yahoo.com,NaN,374,8726,1
18,49,R,debit,NaN,iOS,188,8159,0
21,1907,C,debit,anon.com,NaN,19,8791,1
...,...,...,...,...,...,...,...,...
967,541,W,credit,NaN,NaN,81,7908,0
974,12,C,debit,yahoo.com,NaN,67,8185,0
982,3001,H,debit,anon.com,NaN,389,1621,0
987,1975,W,debit,hotmail.com,NaN,453,2830,1


In [187]:
df_fraud = df_fraud.dropna()

In [188]:
df_fraud.isna().sum()

TransactionAmt        0
ProductCD             0
Card_Type             0
P_emaildomain         0
Device_Info           0
Distance_From_Home    0
Lucky_User_ID         0
is_Fraud              0
dtype: int64

In [191]:
df_fraud.duplicated().sum()

np.int64(0)

In [192]:
df_fraud

,TransactionAmt,ProductCD,Card_Type,P_emaildomain,Device_Info,Distance_From_Home,Lucky_User_ID,is_Fraud
0,3592,H,credit,gmail.com,iOS,15,1189,0
1,3464,H,credit,anon.com,Windows,402,6339,1
2,1356,C,debit,hotmail.com,Mac,52,4895,0
3,4070,W,credit,yahoo.com,Windows,111,8526,1
4,1603,W,debit,hotmail.com,Windows,481,4017,1
...,...,...,...,...,...,...,...,...
995,2661,W,credit,anon.com,Windows,411,8520,1
996,1719,H,credit,hotmail.com,iOS,29,6412,0
997,4063,R,debit,hotmail.com,iOS,441,8797,1
998,3468,W,debit,hotmail.com,Mac,218,5308,1


In [204]:
categorical = df_fraud.select_dtypes('object').columns
categorical

Index(['ProductCD', 'Card_Type', 'P_emaildomain', 'Device_Info'], dtype='object')

In [232]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

X = df_fraud.drop('is_Fraud', axis=1)
y = df_fraud['is_Fraud']

le = LabelEncoder()

for i in categorical:
    X[i] = le.fit_transform(X[i])

X

,TransactionAmt,ProductCD,Card_Type,P_emaildomain,Device_Info,Distance_From_Home,Lucky_User_ID
0,3592,1,0,1,3,15,1189
1,3464,1,0,0,2,402,6339
2,1356,0,1,2,0,52,4895
3,4070,4,0,3,2,111,8526
4,1603,4,1,2,2,481,4017
...,...,...,...,...,...,...,...
995,2661,4,0,0,2,411,8520
996,1719,1,0,2,3,29,6412
997,4063,2,1,2,3,441,8797
998,3468,4,1,2,0,218,5308


In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [274]:
scaler = MinMaxScaler()

x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

x_test_scaled


array([[0.43351438, 0.75      , 0.        , ..., 0.        , 0.02409639,
        0.08857366],
       [0.50573325, 0.        , 0.        , ..., 0.66666667, 0.13453815,
        0.79906177],
       [0.29370348, 0.75      , 0.        , ..., 0.33333333, 0.89156627,
        0.73830001],
       ...,
       [0.49909475, 0.        , 0.        , ..., 1.        , 0.28514056,
        0.46274992],
       [0.74532287, 0.5       , 1.        , ..., 1.        , 0.85140562,
        0.23645705],
       [0.67229934, 0.25      , 0.        , ..., 0.33333333, 0.73895582,
        0.32547749]], shape=(152, 7))

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

filter = SelectKBest(score_func=chi2, k=3)
x_train_filter = filter.fit_transform(x_train_scaled, y_train)
x_test_filter = filter.transform(x_test_scaled)

filter_indeces = filter.get_support(indices=True)
x_train.columns[filter_indeces]

Index(['Tenure', 'SeniorCitizen', 'Dependents'], dtype='object')

In [275]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

lr = LogisticRegression(solver='liblinear')
rfe = RFE(estimator=lr, n_features_to_select=3)

x_train_rfe = rfe.fit_transform(x_train_scaled, y_train)
x_test_rfe  = rfe.transform(x_test_scaled)

rfe_cols = rfe.get_support(indices=True)
x_train.columns[rfe_cols]

Index(['Tenure', 'SeniorCitizen', 'Dependents'], dtype='object')

In [289]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train_scaled, y_train)

importance = rf.feature_importances_
important_feature_indices = np.argsort(importance)[::-1][:3]

x_train.columns[important_feature_indices]

Index(['Tenure', 'Dependents', 'PhoneService'], dtype='object')